## Преобразование ответа от модели в Spans  с соответствующим токенром

In [365]:
import numpy as np
from collections import Counter
from deeppavlov.models.tokenizers.nltk_moses_tokenizer import NLTKMosesTokenizer

In [366]:
# token_text = ['[CLS]', 'субподряд', '##чик', 'северное', 'лине', '##иное', 'производственное', 'управление', 'магистральных', 'газопроводов', '(', 'северное', 'л', '##пу', '##м', '##г', ')', '631', '##22', '##64', '##35', '##88', '##2', 'город', 'москв', '##а', 'улица', 'уса', '##чева', 'дом', '24', 'филиал', 'банка', 'плате', '##ль', '##щик', 'о', '##оо', 'банк', '"', 'сарат', '##ов', '"', 'тел', '.', '8', '344', '741', '908', '##3', 'бик', '[SEP]']

token_text = ['[CLS]', 'ип', 'гер', '##аси', '##нюк', 'иб', '##ра', '##им', 'нас', '##рт', '##динович', '001', '##12', '##94', '##0', '920', '##400', '##04', '##92', '/', '920', '##401', '##00', '##i', '407', '##02', '##81', '##0', '##500', '##0000', '##0', '##75', '##63', 'о', '##оо', '"', 'лит', '##лтон', '"', '№', '##в', '##нг', '##тва', '##т', '=', '=', '299', '##01', '##1', 'г', '.', 'севаст', '##ополь', 'ул', '.', 'адмирала', 'октябрь', '##ского', 'д', '.', '13', 'офис', '1', 'тел', '(', '06', '##92', ')', '671', '##71', '##0', '‚', 'в', 'банк', '"', 'та', '##атта', '"', 'а', '##о', 'г', '.', 'иркут', '##ск', 'бик', '049', '##80', '##57', '##0', '##9', 'кор', '.', 'счет', '301', '##01', '##81', '##0', '##3000', '##0000', '##0', '##70', '##9', 'по', 'ок', '##по', '[SEP]']
# indices = [1, 0, 0, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 7, 9, 7, 7, 9, 7, 5, 6, 6, 6, 6, 6, 6, 6, 0, 0, 0, 0, 0, 3, 3, 4, 4, 4, 4, 4, 0, 0, 11, 12, 12, 12, 12, 0, 0]
indices = [1, 3, 1, 1, 1, 2, 2, 2, 2, 2, 2, 7, 7, 7, 7, 7, 7, 7, 7, 0, 7, 7, 7, 7, 9, 9, 7, 9, 7, 7, 7, 9, 9, 3, 3, 4, 4, 6, 4, 4, 4, 6, 6, 0, 6, 6, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 0, 11, 11, 12, 12, 12, 12, 12, 0, 6, 0, 4, 4, 4, 4, 3, 3, 4, 4, 4, 6, 0, 11, 11, 11, 11, 11, 0, 0, 0, 9, 9, 9, 9, 9, 9, 9, 9, 9, 0, 0, 0, 0]

label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-INNKPP', 'I-INNKPP', 'B-RSKS', 'I-RSKS', 'B-STAT', 'I-STAT']


# text ='Субподрядчик СЕВЕРНОЕ ЛИНЕЙНОЕ ПРОИЗВОДСТВЕННОЕ УПРАВЛЕНИЕ МАГИСТРАЛЬНЫХ ГАЗОПРОВОДОВ (СЕВЕРНОЕ ЛПУМГ) '\
#         '631226435882 ГОРОД МОСКВА УЛИЦА УСАЧЁВА ДОМ 24 Филиал банка Плательщик ООО Банк "Саратов" тел. 8 344 741 9083 БИК'

text = '00112940 9204000492/920401001 ООО "ЛИТЛТОН"  299011 г. Севастополь ул. Адмирала Октябрьского д. 13 офис 1 тел (0692) 671710 ‚ р/с 40702810500000007563 в Банк "ТААТТА"АО г. Иркутск БИК 049805709 кор. счет 30101810300000000709 по ОКПО'

In [367]:
def normalize_tokens(spans):
    """
    Приводим к единому значению тэги у слова, если оно состоит из нескольких токенов и нескольких тэгов
    """
    tokens, entities = zip(*spans)
    new_entities = []
    for i, enti in enumerate(entities):
        if i == 0:
            new_entities.append(Counter(list(filter(lambda x: x[0] == 'B', enti))).most_common(1)[0][0])
        else:
            if len(enti) > 1:
                if len(enti) > 1:              
                    if new_entities[i-1][0][2:] in enti[0] and enti[0][0] == 'I':
                        new_entities.append(enti[0])
                    else:                             
                        new_entities.append(Counter(enti).most_common(1)[0][0])
            else:
                new_entities.append(enti[0])
    return list(zip(tokens,new_entities))

def join_i_tag(spans):
    """
    Объединяем последовательно идущие Тэги I-
    """
    tokens, entities = zip(*spans)
    st = ['', '']
    ent = []
    result = []
    for i, enti in enumerate(entities):
        e0 = enti[0]
        if e0 in ['B', 'O']:
            if len(st[0]) > 0:
                st[0] = st[0].strip()
                result.append(st)
                st = ['', '']
            result.append([tokens[i], enti])
            continue
        if 0 < i < len(entities)-1:
            if e0 in ['I']:
                # if len(st) > 0:
                if entities[i-1] == enti:
                    st[0] += ' '+tokens[i]
                    st[1] = enti
                    continue
                elif entities[i+1] == enti:
                    st[0] += ' '+tokens[i]
                    st[1] = enti
                    continue
                else:
                    if len(st[0]) > 0:
                        st[0] = st[0].strip()
                        result.append(st)
                        st = ['', '']
    return result

def join_bi_tag(spans):
    tokens, entities = zip(*spans)
    st = ['', '']
    ent = []
    result = []
    for i, enti in enumerate(entities):
        e0 = enti[0]
        if e0 in ['B']:
            if entities[i+1][0] == 'I':
                if entities[i+1][2:] == enti[2:]:
                    st[0] += ' ' + tokens[i]
                    st[1] = enti
                    continue
            else:
                if len(st[0]) > 0:
                    if st[1][2:] == enti[2:]:
                        st[0] += ' ' + tokens[i]
                        result.append(st)
                        st = ['', '']
                else:
                    result.append([tokens[i], enti])
        elif e0 in ['I']:
            if len(st[0]) > 0:
                if st[1][2:] == enti[2:]:
                    st[0] += ' ' + tokens[i]
                    result.append(st)
                    st = ['', '']
                else:
                    st[0] = st[0].strip()
                    result.append(st)
                    st = ['', '']
                continue
            else:
                if entities[i+1][0] == 'B':
                    if entities[i+1][2:] == enti[2:]:
                        st[0] += ' ' + tokens[i]
                        st[1] = entities[i+1] #enti
                        continue

        elif e0 in ['O']:
            result.append([tokens[i], enti])
            continue
    return result

def get_single_tag(tags=[]):
    """
    Предварительный выбор тэга
    """
    if len(set(tags)) == 1:
        return [tags[0]] if tags[0] != 'O' else [tags[0]]
    if 'O' in tags:
        return ['O']
    return tags #list(map(lambda x: x[2:] if x[0] != 'O' else x[0], tags)) #Counter(tags).most_common(1)[0][0]


def join_tokens(token_text, indices):
    """
    Объединяем токены в слова и соединяем с тэгами
    """
    conj = '##'
    sp = ""
    spans = []
    tags = []
    tag = []
    i = 0
    
    for t, idx in zip(token_text, indices):
        if t not in ['[CLS]', '[SEP]']:
            if t[:2] == conj:
                sp += t[2:]
                tags.append(label_list[idx])
            else:
                sp += f' {t}'
                tags.append(label_list[idx])
            if i < len(token_text)-1:
                if token_text[i+1][:2] != conj:
                    spans.append([sp.strip(), get_single_tag(tags)])
                    sp = ""
                    tags = []
    #         print(f'{t:15s} {label_list[idx]:10s}')
        i += 1
    return spans

def get_final_tags(spans):
    """
    Финальное объединение тэгов и токенов, убирание B-I- префиксов
    """
    tokens, entities = zip(*spans)
    st = ""
    ent = []
    result = []
    i = -1
    detokenizer = NLTKMosesTokenizer('ru').detokenizer
    while i < len(entities)-1:
        i+=1
        enti = entities[i]
        e0 = enti[0]
    #     print(enti)
        if enti == 'B-ORG' and (len(regex.findall(r"^(\bиндивидуальный\b){e<=4}", tokens[i]))>0 or tokens[i]=='ип'):
            if i < len(entities)-2:
                if entities[i+1] == 'B-PER':
                    st = tokens[i]+tokens[i+1] 
                    result.append([enti[2:], detokenizer.detokenize([''.join(c)  for c in st.split()])])
                    i+=1
        else:
            if i < len(entities)-2:
                if len(tokens[i]) <= 2 and enti == entities[i+1]:
                    st = tokens[i]+tokens[i+1] 
                    result.append([enti[2:] if enti != 'O' else 'O', detokenizer.detokenize([''.join(c)  for c in st.split()])])
                    i+=1            
                else:
                    result.append([enti[2:] if enti != 'O' else 'O', detokenizer.detokenize([''.join(c)  for c in tokens[i].split()])])
            else:
                result.append([enti[2:] if enti != 'O' else 'O', detokenizer.detokenize([''.join(c)  for c in tokens[i].split()])])
    return result

In [206]:
join_tokens(token_text, indices)

[['ип', ['B-ORG']],
 ['герасинюк', ['B-PER']],
 ['ибраим', ['I-PER']],
 ['насртдинович', ['I-PER']],
 ['00112940', ['B-INNKPP']],
 ['9204000492', ['B-INNKPP']],
 ['/', ['O']],
 ['92040100i', ['B-INNKPP']],
 ['40702810500000007563',
  ['B-RSKS',
   'B-RSKS',
   'B-INNKPP',
   'B-RSKS',
   'B-INNKPP',
   'B-INNKPP',
   'B-INNKPP',
   'B-RSKS',
   'B-RSKS']],
 ['ооо', ['B-ORG']],
 ['"', ['I-ORG']],
 ['литлтон', ['I-ORG', 'I-LOC']],
 ['"', ['I-ORG']],
 ['№внгтват', ['O']],
 ['=', ['I-LOC']],
 ['=', ['I-LOC']],
 ['299011', ['B-LOC']],
 ['г', ['B-LOC']],
 ['.', ['I-LOC']],
 ['севастополь', ['I-LOC']],
 ['ул', ['I-LOC']],
 ['.', ['I-LOC']],
 ['адмирала', ['I-LOC']],
 ['октябрьского', ['I-LOC']],
 ['д', ['I-LOC']],
 ['.', ['I-LOC']],
 ['13', ['I-LOC']],
 ['офис', ['I-LOC']],
 ['1', ['I-LOC']],
 ['тел', ['O']],
 ['(', ['B-STAT']],
 ['0692', ['B-STAT', 'I-STAT']],
 [')', ['I-STAT']],
 ['671710', ['I-STAT']],
 ['‚', ['O']],
 ['в', ['I-LOC']],
 ['банк', ['O']],
 ['"', ['I-ORG']],
 ['таатта', ['I-O

In [309]:
# получаем пары: слово-токены
join_tokens(token_text, indices)

[['ип', ['B-ORG']],
 ['герасинюк', ['B-PER']],
 ['ибраим', ['I-PER']],
 ['насртдинович', ['I-PER']],
 ['00112940', ['B-INNKPP']],
 ['9204000492', ['B-INNKPP']],
 ['/', ['O']],
 ['92040100i', ['B-INNKPP']],
 ['40702810500000007563',
  ['B-RSKS',
   'B-RSKS',
   'B-INNKPP',
   'B-RSKS',
   'B-INNKPP',
   'B-INNKPP',
   'B-INNKPP',
   'B-RSKS',
   'B-RSKS']],
 ['ооо', ['B-ORG']],
 ['"', ['I-ORG']],
 ['литлтон', ['I-ORG', 'I-LOC']],
 ['"', ['I-ORG']],
 ['№внгтват', ['O']],
 ['=', ['I-LOC']],
 ['=', ['I-LOC']],
 ['299011', ['B-LOC']],
 ['г', ['B-LOC']],
 ['.', ['I-LOC']],
 ['севастополь', ['I-LOC']],
 ['ул', ['I-LOC']],
 ['.', ['I-LOC']],
 ['адмирала', ['I-LOC']],
 ['октябрьского', ['I-LOC']],
 ['д', ['I-LOC']],
 ['.', ['I-LOC']],
 ['13', ['I-LOC']],
 ['офис', ['I-LOC']],
 ['1', ['I-LOC']],
 ['тел', ['O']],
 ['(', ['B-STAT']],
 ['0692', ['B-STAT', 'I-STAT']],
 [')', ['I-STAT']],
 ['671710', ['I-STAT']],
 ['‚', ['O']],
 ['в', ['I-LOC']],
 ['банк', ['O']],
 ['"', ['I-ORG']],
 ['таатта', ['I-O

#### Pipe по получению Spans

In [273]:
# получаем пары: слово-токены
spans = join_tokens(token_text, indices)
# выбираем нужный токен для слова
spans = normalize_tokens(spans)
spans = join_i_tag(spans)
spans = join_bi_tag(spans)
# join_bi_tag(spans)

In [280]:
len(entities)

25

In [438]:
spans = [('ооо', 'B-ORG'), ('ромашка', 'I-ORG'), ('-', 'I-ORG'), ('профиль', 'O'), ('00112940', 'B-INNKPP'), ('9204000492', 'B-INNKPP'), ('/', 'O'), 
         ('92040', 'B-INNKPP'), ('|', 'O'), ('00i', 'I-ORG'), ('40702810500000007563', 'B-INNKPP'), ('ооо', 'B-ORG'), ('литлтон', 'I-ORG'), 
         ('№внгтват', 'I-ORG'), ('=', 'I-ORG'), ('=', 'I-LOC'), ('299011', 'B-LOC'), ('г', 'B-LOC'), ('.', 'I-LOC'), ('севастополь', 'I-LOC'), ('ул', 'I-LOC'), 
         ('.', 'I-LOC'), ('адмирала', 'I-LOC'), ('октябрьского', 'I-LOC'), ('д', 'I-LOC'), ('.', 'I-LOC'), ('13', 'I-LOC'), ('офис', 'I-LOC'), ('1', 'I-LOC'), 
         ('тел', 'O'), ('(', 'B-STAT'), ('0692', 'B-STAT'), (')', 'I-STAT'), ('671710', 'I-STAT'), ('‚', 'I-LOC'), ('в', 'I-LOC'), ('банк', 'O'), ('"', 'I-ORG'), 
         ('таатта', 'I-ORG'), ('"', 'I-ORG'), (',', 'I-LOC'), ('ао', 'I-ORG'), ('г', 'I-ORG'), ('.', 'I-ORG'), ('иркутск', 'I-ORG'), ('бик', 'O'), 
         ('049805709', 'B-STAT'), ('кор', 'O'), ('.', 'O'), ('счет', 'O'), ('30101810300000000709', 'B-RSKS'), ('по', 'O'), ('окпо', 'O')]

In [445]:
tokens, entities = zip(*spans)
st = ['', '']
ent = []
result = []

for i, enti in enumerate(entities):
    enti = entities[i]
    e0 = enti[0]
    
    if e0 in ['I', 'O']:
        if st[1] == enti:
            st[0] += f' {tokens[i]}'
            st[1] = enti
        else:
            if len(st[0]) > 0:
                st[0] = st[0].strip()
                result.append(st)
            st = [tokens[i], enti]
    #     st = [tokens[i].strip(), enti]
    if e0 in ['B']:
        # st = [tokens[i].strip(), enti]
        if len(st[0]) > 0:
            result.append(st)
            st = [tokens[i], enti]
        else:
            result.append([tokens[i].strip(), enti])
if len(st[0]) > 0 and st[1] != result[-1][0]:
    result.append(st)

In [409]:
tokens[52]

'окпо'

In [446]:
print(result)

[['ооо', 'B-ORG'], ['ромашка -', 'I-ORG'], ['профиль', 'O'], ['00112940', 'B-INNKPP'], ['9204000492', 'B-INNKPP'], ['/', 'O'], ['92040', 'B-INNKPP'], ['|', 'O'], ['00i', 'I-ORG'], ['40702810500000007563', 'B-INNKPP'], ['ооо', 'B-ORG'], ['литлтон №внгтват =', 'I-ORG'], ['=', 'I-LOC'], ['299011', 'B-LOC'], ['г', 'B-LOC'], ['. севастополь ул . адмирала октябрьского д . 13 офис 1', 'I-LOC'], ['тел', 'O'], ['(', 'B-STAT'], ['0692', 'B-STAT'], [') 671710', 'I-STAT'], ['‚ в', 'I-LOC'], ['банк', 'O'], ['" таатта "', 'I-ORG'], [',', 'I-LOC'], ['ао г . иркутск', 'I-ORG'], ['бик', 'O'], ['049805709', 'B-STAT'], ['кор . счет', 'O'], ['30101810300000000709', 'B-RSKS'], ['по окпо', 'O']]


In [463]:
spans = [['ооо', 'B-ORG'], ['ромашка -', 'I-ORG'], ['профиль', 'O'], ['00112940', 'B-INNKPP'], ['9204000492', 'B-INNKPP'], ['/', 'O'], 
         ['92040', 'B-INNKPP'], ['|', 'O'], ['00i', 'I-ORG'], ['40702810500000007563', 'B-INNKPP'], ['ооо', 'B-ORG'], ['литлтон №внгтват =', 'I-ORG'], 
         ['=', 'I-LOC'], ['299011', 'B-LOC'], ['г', 'B-LOC'], ['. севастополь ул . адмирала октябрьского д . 13 офис 1', 'I-LOC'], ['тел', 'O'], 
         ['(', 'B-STAT'], ['0692', 'B-STAT'], [') 671710', 'I-STAT'], ['‚ в', 'I-LOC'], ['банк', 'O'], ['" таатта "', 'I-ORG'], [',', 'I-LOC'], 
         ['ао г . иркутск', 'I-ORG'], ['бик', 'O'], 
         ['049805709', 'B-STAT'], ['кор . счет', 'O'], ['30101810300000000709', 'B-RSKS'], ['по окпо', 'O']]

tokens, entities = zip(*spans)
st = ['', '']
ent = []
result = []

for i, enti in enumerate(entities):
    e0 = enti[0]
    if e0 == 'I':
        if i == 0:
            result.append([tokens[i], enti.replace('I-', 'B-')])
        else:
            e00 = entities[i-1][0]
            if e00 in ['I', 'O'] and entities[i-1] != enti:
                result.append([tokens[i], enti.replace('I-', 'B-')])
            else:
                result.append([tokens[i], enti])
    else:
        result.append([tokens[i], enti.replace('I-', 'B-')])


tokens, entities = zip(*result)
st = ['', '']
ent = []
result = []

for i, enti in enumerate(entities):
    e0 = enti[0]
    if e0 in ['B']:
        if entities[i + 1][0] == 'I':
            if entities[i + 1][2:] == enti[2:]:
                st[0] += ' ' + tokens[i]
                st[1] = enti
                continue
        else:
            if len(st[0]) > 0:
                if st[1][2:] == enti[2:]:
                    st[0] += ' ' + tokens[i]
                    result.append(st)
                    st = ['', '']
            else:
                result.append([tokens[i], enti])
    elif e0 in ['I']:
        if len(st[0]) > 0:
            if st[1][2:] == enti[2:]:
                st[0] += ' ' + tokens[i]
                result.append(st)
                st = ['', '']
            else:
                st[0] = st[0].strip()
                result.append(st)
                st = ['', '']
            continue
        else:
            if entities[i + 1][0] == 'B':
                if entities[i + 1][2:] == enti[2:]:
                    st[0] += ' ' + tokens[i]
                    st[1] = entities[i + 1]  # enti
                    continue

    elif e0 in ['O']:
        result.append([tokens[i], enti])
        continue

---  ооо B-ORG
ромашка - I-ORG
---  | O
00i I-ORG
++++  | O
00i I-ORG
---  ооо B-ORG
литлтон №внгтват = I-ORG
---  литлтон №внгтват = I-ORG
= I-LOC
++++  литлтон №внгтват = I-ORG
= I-LOC
---  г B-LOC
. севастополь ул . адмирала октябрьского д . 13 офис 1 I-LOC
---  0692 B-STAT
) 671710 I-STAT
---  ) 671710 I-STAT
‚ в I-LOC
++++  ) 671710 I-STAT
‚ в I-LOC
---  банк O
" таатта " I-ORG
++++  банк O
" таатта " I-ORG
---  " таатта " I-ORG
, I-LOC
++++  " таатта " I-ORG
, I-LOC
---  , I-LOC
ао г . иркутск I-ORG
++++  , I-LOC
ао г . иркутск I-ORG


In [464]:
result

[[' ооо ромашка -', 'B-ORG'],
 ['профиль', 'O'],
 ['00112940', 'B-INNKPP'],
 ['9204000492', 'B-INNKPP'],
 ['/', 'O'],
 ['92040', 'B-INNKPP'],
 ['|', 'O'],
 ['00i', 'B-ORG'],
 ['40702810500000007563', 'B-INNKPP'],
 [' ооо литлтон №внгтват =', 'B-ORG'],
 ['=', 'B-LOC'],
 ['299011', 'B-LOC'],
 [' г . севастополь ул . адмирала октябрьского д . 13 офис 1', 'B-LOC'],
 ['тел', 'O'],
 ['(', 'B-STAT'],
 [' 0692 ) 671710', 'B-STAT'],
 ['‚ в', 'B-LOC'],
 ['банк', 'O'],
 ['" таатта "', 'B-ORG'],
 [',', 'B-LOC'],
 ['ао г . иркутск', 'B-ORG'],
 ['бик', 'O'],
 ['049805709', 'B-STAT'],
 ['кор . счет', 'O'],
 ['30101810300000000709', 'B-RSKS'],
 ['по окпо', 'O']]

In [432]:
Counter([]).most_common(1)

[]

In [5]:
res_spans = [{'word': 'Ромашка', 'coord': [0, 7], 'tags': ['B-ORG', 0.9468191862106323]}, {'word': ', ООО', 'coord': [7, 12], 'tags': ['I-ORG', 0.9496479034423828]}, {'word': 'ООО', 'coord': [13, 16], 'tags': ['B-ORG', 0.9864256381988525]}, {'word': '" Гарант - СВ "', 'coord': [17, 28], 'tags': ['I-ORG', 0.9997937083244324]}]
st = {'word': '', 'coord': [], 'tags': ['', 0]}
result = []

for i, enti in enumerate(res_spans):
    e0 = enti['tags'][0][0]
    if e0 in ['B']:
        if i < len(res_spans) - 1 and res_spans[i + 1]['tags'][0][0] == 'I':
            if res_spans[i + 1]['tags'][0][2:] == enti['tags'][0][2:]:
                st['word'] += ' ' + enti['word']
                if len(st['coord']) > 0:
                    st['coord'] = [st['coord'][0], enti['coord'][1]]
                else:
                    st['coord'] = [enti['coord'][0], enti['coord'][1]]
                st['tags'] = [enti['tags'][0].split('-')[-1], enti['tags'][1]] #enti['tags']
                continue
        else:
            if len(st['word']) > 0:
                if st['tags'][0][2:] == enti[2:]:
                    st['word'] += ' ' + enti['word']
                    if len(st['coord']) > 0:
                        st['coord'] = [st['coord'][0], enti['coord'][1]]
                    else:
                        st['coord'] = [enti['coord'][0], enti['coord'][1]]
                    st['tags'] = [enti['tags'][0].split('-')[-1], enti['tags'][1]] #enti['tags']
                    st = {'word': '', 'coord': [], 'tags': ['', 0]}
            else:
                result.append(enti)
    elif e0 in ['I']:
        if len(st['word']) > 0:
            if st['tags'][0][2:] == enti['tags'][0][2:]:
                st['word'] += ' ' + enti['word']
                if len(st['coord']) > 0:
                    st['coord'] = [st['coord'][0], enti['coord'][1]]
                else:
                    st['coord'] = [enti['coord'][0], enti['coord'][1]]
                #                 st['tags'] = enti['tags']
                result.append(st)
                st = {'word': '', 'coord': [], 'tags': ['', 0]}
            else:
                st['word'] = st['word'].strip()
                result.append(st)
                st = {'word': '', 'coord': [], 'tags': ['', 0]}
            continue
        else:
            if i <= len(res_spans) - 2:
                if res_spans[i + 1]['tags'][0][0] == 'B':
                    if res_spans[i + 1]['tags'][0][2:] == enti['tags'][0][2:]:
                        st['word'] += ' ' + enti['word']

                        if len(st['coord']) > 0:
                            st['coord'] = [st['coord'][0], res_spans[i + 1]['coord'][1]]
                        else:
                            st['coord'] = [res_spans[i + 1]['coord'][0], res_spans[i + 1]['coord'][1]]
                        st['tags'] = [enti['tags'][0].split('-')[-1], enti['tags'][1]] #enti['tags']
                        st['tags'] = [res_spans[i + 1][0].split('-')[-1], res_spans[i + 1][1]]#res_spans[i + 1]['tags']
                        continue
            else:
                if res_spans[i - 1]['tags'][0][2:] == enti['tags'][0][2:]:
                    st['word'] += ' ' + enti['word']

                    if len(st['coord']) > 0:
                        st['coord'] = [st['coord'][0], res_spans[i]['coord'][1]]
                    else:
                        st['coord'] = [res_spans[i - 1]['coord'][0], res_spans[i]['coord'][1]]

                    st['tags'] = [res_spans[i - 1][0].split('-')[-1], res_spans[i - 1][1]]#res_spans[i - 1]['tags']
                    result.append(st)
                    st = {'word': '', 'coord': [], 'tags': ['', 0]}
                    continue
                else:
                    st['word'] = enti['word']
                    st['tags'] = [enti['tags'][0].split('-')[-1], enti['tags'][1]] #enti['tags']
                    st['coord'] = [res_spans[i]['coord'][0], res_spans[i]['coord'][1]]
                    result.append(st)
                    st = {'word': '', 'coord': [], 'tags': ['', 0]}
                    print(res_spans[i])

    elif e0 in ['O']:
        result.append(enti)
        continue
result

[{'word': 'Ромашка', 'coord': [0, 7], 'tags': ['ORG', 0.9468191862106323]},
 {'word': 'ООО', 'coord': [13, 16], 'tags': ['ORG', 0.9864256381988525]}]

In [3]:
res_spans =[{'word': 'Ромашка', 'coord': [0, 7], 'tags': ['B-ORG', 0.9468191862106323]}, 
             {'word': ', ООО', 'coord': [7, 12], 'tags': ['I-ORG', 0.9496479034423828]}, 
             {'word': 'ООО', 'coord': [13, 16], 'tags': ['B-ORG', 0.9864256381988525]}, 
             {'word': '" Гарант - СВ "', 'coord': [17, 28], 'tags': ['I-ORG', 0.9997937083244324]}]

st = {'word': '', 'coord': [], 'tags': ['', 0]}
result = []

for i, enti in enumerate(res_spans):
    e0 = enti['tags'][0][0]
    if e0 in ['B']:
        if i < len(res_spans) - 1 and res_spans[i + 1]['tags'][0][0] == 'I':
            if res_spans[i + 1]['tags'][0][2:] == enti['tags'][0][2:]:
                st['word'] += ' ' + enti['word']
                if len(st['coord']) > 0:
                    st['coord'] = [st['coord'][0], enti['coord'][1]]
                else:
                    st['coord'] = [enti['coord'][0], enti['coord'][1]]
                st['tags'] = enti['tags'].split()
                continue
        else:
            if len(st['word']) > 0:
                if st['tags'][0][2:] == enti[2:]:
                    st['word'] += ' ' + enti['word']
                    if len(st['coord']) > 0:
                        st['coord'] = [st['coord'][0], enti['coord'][1]]
                    else:
                        st['coord'] = [enti['coord'][0], enti['coord'][1]]
                    st['tags'] = enti['tags']
                    st = {'word': '', 'coord': [], 'tags': ['', 0]}
            else:
                result.append(enti)
    elif e0 in ['I']:
        if len(st['word']) > 0:
            if st['tags'][0][2:] == enti['tags'][0][2:]:
                st['word'] += ' ' + enti['word']
                if len(st['coord']) > 0:
                    st['coord'] = [st['coord'][0], enti['coord'][1]]
                else:
                    st['coord'] = [enti['coord'][0], enti['coord'][1]]
                #                 st['tags'] = enti['tags']
                result.append(st)
                st = {'word': '', 'coord': [], 'tags': ['', 0]}
            else:
                st['word'] = st['word'].strip()
                result.append(st)
                st = {'word': '', 'coord': [], 'tags': ['', 0]}
            continue
        else:
            if i <= len(res_spans) - 2:
                if res_spans[i + 1]['tags'][0][0] == 'B':
                    if res_spans[i + 1]['tags'][0][2:] == enti['tags'][0][2:]:
                        st['word'] += ' ' + enti['word']

                        if len(st['coord']) > 0:
                            st['coord'] = [st['coord'][0], res_spans[i + 1]['coord'][1]]
                        else:
                            st['coord'] = [res_spans[i + 1]['coord'][0], res_spans[i + 1]['coord'][1]]
                        st['tags'] = enti['tags']
                        st['tags'] = res_spans[i + 1]['tags']
                        continue
            else:
                if res_spans[i - 1]['tags'][0][2:] == enti['tags'][0][2:]:
                    st['word'] += ' ' + enti['word']

                    if len(st['coord']) > 0:
                        st['coord'] = [st['coord'][0], res_spans[i]['coord'][1]]
                    else:
                        st['coord'] = [res_spans[i - 1]['coord'][0], res_spans[i]['coord'][1]]

                    st['tags'] = res_spans[i - 1]['tags']
                    result.append(st)
                    st = {'word': '', 'coord': [], 'tags': ['', 0]}
                    continue
                else:
                    st['word'] = enti['word']
                    st['tags'] = enti['tags']
                    st['coord'] = [res_spans[i]['coord'][0], res_spans[i]['coord'][1]]
                    result.append(st)
                    st = {'word': '', 'coord': [], 'tags': ['', 0]}
                    print(res_spans[i])

    elif e0 in ['O']:
        result.append(enti)
        continue
        
result

[{'word': ' Ромашка , ООО',
  'coord': [0, 12],
  'tags': ['B-ORG', 0.9468191862106323]},
 {'word': ' ООО " Гарант - СВ "',
  'coord': [13, 28],
  'tags': ['B-ORG', 0.9864256381988525]}]

In [572]:
def get_most_common(enti):
    """
    Считаем наиболее частые встречи тегов в зависимости от приставки тега и в общем
    """
    result = dict(B=[], I=[], O=[], Overall=[])
    cnt = Counter(list(filter(lambda x: x[0] == 'B', enti))).most_common()
    max_v = cnt[0][1] if len(cnt) > 0 else 0
    result['B'] = list(filter(lambda x: x[1] >= max_v, cnt)) if max_v > 0 else [('', 0), ]

    cnt = Counter(list(filter(lambda x: x[0] == 'I', enti))).most_common()
    max_v = cnt[0][1] if len(cnt) > 0 else 0
    result['I'] = list(filter(lambda x: x[1] >= max_v, cnt)) if max_v > 0 else [('', 0), ]

    cnt = Counter(list(filter(lambda x: x[0] == 'O', enti))).most_common()
    max_v = cnt[0][1] if len(cnt) > 0 else 0
    result['O'] = list(filter(lambda x: x[1] >= max_v, cnt)) if max_v > 0 else [('', 0), ]

    cnt = Counter(enti).most_common()
    max_v = cnt[0][1] if len(cnt) > 0 else 0
    result['Overall'] = list(filter(lambda x: x[1] >= max_v, cnt)) if max_v > 0 else [('', 0), ]
    return result


def normalize_tokens(joined_tokens):
    """
    Приводим к единому значению тэги у слова, если оно состоит из нескольких токенов и нескольких тэгов
    """

    assert (joined_tokens is not None), "joined_tokens  еще не заполнена, сначала вызовите метод join_tokens()"

    spans = joined_tokens
    tokens, entities = zip(*spans)
    new_entities = []
    for i, enti in enumerate(entities):
        cnt = get_most_common(enti)
#   cnt = {'B': [('B-LOC', 2)],
#          'I': [('I-LOC', 1)],
#          'O': [('O', 1)],
#          'Overall': [('B-LOC', 2)]}
        if 'литлтон' in tokens[i]:
            print(tokens[i])
        if i == 0:
            
            if len(cnt['Overall']) == 1:
                new_entities.append([tokens[i], cnt['Overall'][0][0].replace('I-', 'B-')])
            elif len(cnt['Overall']) > 1:
                
                cnt_1 = get_most_common(entities[i+1])
                new_ent = ""
                for c in cnt['Overall']:
                    t0 = c[0].replace('I-', '').replace('B-', '')
                    for cc in cnt_1['Overall']:
                        tt0 = cc[0].replace('I-', '').replace('B-', '')
                        if t0 == tt0:
                            
                            new_ent = c[0].replace('I-', 'B-')
                            print(t0, ' - ', tt0, ' - ', new_ent)
                            break
                if new_ent != "":
                    new_entities.append([tokens[i], new_ent])
                else:
                    if cnt['B'][0][1] > 0:
                        new_entities.append([tokens[i], cnt['B'][0][0]])
                    elif cnt['I'][0][1] > 0:
                        new_entities.append([tokens[i], cnt['I'][0][0]])
                    else:
                        new_entities.append([tokens[i], 'O'])
#         else:
#             if enti[0] == "O":
#                 new_entities.append([tokens[i], enti[0]])
#             else:
#                 if i < len(entities)-1:
#                     t00 = new_entities[i - 1][1].replace('I-', '').replace('B-', '')
#                     cnt_1 = get_most_common(entities[i + 1])
#                     new_ent = ""
#                     for c in  cnt_1['Overall']:
#                         t0 = c[0].replace('I-', '').replace('B-', '')

#                         if t00 == t0:
#                             print(tokens[i], ' ',t0)
#                             new_ent = f"I-{t00}"
#                     if new_ent != "":
#                         new_entities.append([tokens[i], new_ent])
#                     else:
#                         new_entities.append([tokens[i], enti[0]])
#                 else:
#                     if new_entities[i - 1][1][2:] in enti[0] and enti[0][0] == 'I':
#                         new_entities.append([tokens[i], enti[0]])
#                     else:
#                         new_entities.append([tokens[i], cnt['Overall'][0][0]])


#     norm_tokens = list(zip(tokens, new_entities))
    return  new_entities

In [561]:
get_most_common(['B-RSKS', 'B-RSKS', 'B-RSKS', 'B-RSKS', 'B-RSKS', 'B-RSKS', 'B-RSKS', 'B-RSKS', 'B-RSKS', 'B-RSKS', 'B-INNKPP'])

{'B': [('B-RSKS', 10)],
 'I': [('', 0)],
 'O': [('', 0)],
 'Overall': [('B-RSKS', 10)]}

In [562]:
joined_tokens = [['муп', ['O', 'I-ORG']], ['мп', ['B-ORG', 'I-ORG']], ['городского', ['I-ORG']], ['округа', ['I-ORG']], ['самара', ['I-ORG']], ['красноглинские', ['I-ORG']], ['бани', ['I-ORG']], ['+', ['B-STAT']], ['7', ['B-STAT']], ['-', ['B-STAT']], ['830', ['B-STAT']], ['-', ['B-STAT']], ['199', ['B-STAT']], ['-', ['B-STAT']], ['8031', ['B-STAT', 'B-INNKPP']], ['адрес', ['O']], ['филиала', ['O']], ['банка', ['O']], ['брянская', ['I-ORG', 'B-LOC']], ['область', ['I-LOC']], [',', ['I-LOC']], ['город', ['I-LOC']], ['новозыбков', ['I-LOC']], ['банк', ['I-LOC']], ['инвестор', ['O']], ['инн', ['O']], ['/', ['O']], ['кпп', ['O']], ['100601303468', ['B-INNKPP']], ['/', ['O']], ['632501001', ['B-INNKPP']], ['корр', ['O']], ['.', ['O']], ['счет', ['O']], ['30310411361955797442', ['B-RSKS']]]

In [549]:
joined_tokens = [['орпрпghh', ['O']], ['#', ['O']], ['*', ['O']], ['*', ['O']], ['заказчик', ['O']], ['ооо', ['B-ORG', 'B-LOC']], 
                 
                 ['ромашка', ['I-ORG']], ['-', ['O', 'B-ORG']], ['профиль', ['O', 'I-ORG']], ['кор', ['O']], ['.', ['O']], ['счет', ['O']], 
                 ['30101810300000000709', ['B-INNKPP']], ['9204000492', ['B-INNKPP']], ['/', ['O']], ['92040001', ['B-INNKPP']], 
                 
                 
                 ['"', ['I-ORG']], ['литлтон', ['I-ORG']], ['"', ['I-ORG']], [',', ['I-LOC']], ['ооо', ['I-ORG']], 
                 ['299011', ['B-LOC', 'B-LOC', 'B-INNKPP']], ['г', ['B-LOC']], ['.', ['I-LOC']], 
                 
                 
                 ['севастополь', ['I-LOC']], ['ул', ['I-LOC']], ['.', ['I-LOC']], 
                 ['адмирала', ['I-LOC']], ['октябрьского', ['I-LOC']], ['д', ['I-LOC']], ['.', ['I-LOC']], 
                 ['13', ['I-LOC']], ['офис', ['I-LOC']], ['1', ['I-LOC']], ['тел', ['O']], ['(', ['B-STAT']], 
                 ['0692', ['B-STAT', 'I-STAT']], [')', ['I-STAT']], ['671710', ['I-STAT']], ['‚', ['O']], ['в', ['O']], 
                 ['банк', ['O']], ['вфыssf', ['O']], ['d', ['O']], ['ао', ['B-ORG']],
                 
                 ['"', ['I-ORG']], ['таатта', ['I-ORG']], ['"', ['I-ORG']], [',', ['I-ORG']], ['г', ['I-ORG']], ['.', ['I-LOC']], ['иркутск', ['I-LOC']], ['бик', ['O']],
                 
                 ['049805709', ['B-STAT', 'B-STAT', 'B-STAT', 'B-INNKPP', 'B-RSKS']], ['40702810500000007563', ['B-RSKS']], ['по', ['O']], ['окпо', ['O']]]

In [573]:
normalize_tokens(joined_tokens)

ORG  -  ORG  -  B-ORG


[['муп', 'B-ORG']]

In [16]:
import pandas as pd

l = [['num_ssr eng', [["Ь'Т.д‘‚’08;'П", 78, 32, 205, 55, 57]], [{'Ь': 57, 'b': 50, 'Е': 37, 'H': 15, 'Ё': 73, '?': 74, 'r': 61}, {"'": 107, 'Т': 37, 'T': 15, 'Ь': 73, 'L': 74, 'x': 61}, {'Т': 118, 'T': 50, '.': 52, '#': 147}, {'.': 57, 'J': 50, '#': 37, 'G': 15, 'О': 73, 'O': 74, 'l': 61}, {'д': 57, '!': 50, 'О': 37, 'O': 15, '8': 147, 'l': 61}, {'‘': 57, 'O': 50, 'З': 37, 'S': 15, '#': 147, '0': 61}, {'‚': 57, '8': 50, 'Е': 37, 'G': 15, '1': 73, 'l': 74, '3': 61}, {'’': 57, ';': 50, 'Т': 37, '1': 15, '': 147, 'l': 61}, {'0': 57, '’': 50, '': 199, 'l': 61}, {'8': 57, 'H': 50, '': 199, '1': 61}, {';': 57, '': 310}, {"'": 57, '': 310}, {'П': 57, '': 310}], 'num16.jpg', "?'#O8#l", "?'#O8#l", "?'#O8#l", 'O8#l', 'I_ench_lines'], ['num_ssr rus', [["Ь'Т.д‘‚’08;'П", 78, 32, 205, 55, 57]], [{'Ь': 57, 'b': 50, 'Е': 37, 'H': 15, 'Ё': 73, '?': 74, 'r': 61}, {"'": 107, 'Т': 37, 'T': 15, 'Ь': 73, 'L': 74, 'x': 61}, {'Т': 118, 'T': 50, '.': 52, '#': 147}, {'.': 57, 'J': 50, '#': 37, 'G': 15, 'О': 73, 'O': 74, 'l': 61}, {'д': 57, '!': 50, 'О': 37, 'O': 15, '8': 147, 'l': 61}, {'‘': 57, 'O': 50, 'З': 37, 'S': 15, '#': 147, '0': 61}, {'‚': 57, '8': 50, 'Е': 37, 'G': 15, '1': 73, 'l': 74, '3': 61}, {'’': 57, ';': 50, 'Т': 37, '1': 15, '': 147, 'l': 61}, {'0': 57, '’': 50, '': 199, 'l': 61}, {'8': 57, 'H': 50, '': 199, '1': 61}, {';': 57, '': 310}, {"'": 57, '': 310}, {'П': 57, '': 310}], 'num16.jpg', "?'#O8#l", "?'#O8#l", "?'#O8#l", 'O8#l', 'I_ench_lines'], ['rus --oem 3 --psm 11', [['ЕТ.', 1262, 395, 3073, 1185, 8], ['#', 2842, 391, 3010, 1392, 43], ['ОЗЕТ', 3946, 395, 5697, 1185, 60]], [{'Ь': 57, 'b': 50, 'Е': 37, 'H': 15, 'Ё': 73, '?': 74, 'r': 61}, {"'": 107, 'Т': 37, 'T': 15, 'Ь': 73, 'L': 74, 'x': 61}, {'Т': 118, 'T': 50, '.': 52, '#': 147}, {'.': 57, 'J': 50, '#': 37, 'G': 15, 'О': 73, 'O': 74, 'l': 61}, {'д': 57, '!': 50, 'О': 37, 'O': 15, '8': 147, 'l': 61}, {'‘': 57, 'O': 50, 'З': 37, 'S': 15, '#': 147, '0': 61}, {'‚': 57, '8': 50, 'Е': 37, 'G': 15, '1': 73, 'l': 74, '3': 61}, {'’': 57, ';': 50, 'Т': 37, '1': 15, '': 147, 'l': 61}, {'0': 57, '’': 50, '': 199, 'l': 61}, {'8': 57, 'H': 50, '': 199, '1': 61}, {';': 57, '': 310}, {"'": 57, '': 310}, {'П': 57, '': 310}], 'num16.jpg', "?'#O8#l", "?'#O8#l", "?'#O8#l", 'O8#l', 'I_ench_lines'], ['eng --oem 3 --psm 11', [['HT.', 1262, 395, 3073, 1185, 0], ['G', 2676, 391, 3010, 1392, 0], ['OSG', 3284, 395, 5057, 1185, 1], ['1', 5268, 417, 5697, 1126, 62]], [{'Ь': 57, 'b': 50, 'Е': 37, 'H': 15, 'Ё': 73, '?': 74, 'r': 61}, {"'": 107, 'Т': 37, 'T': 15, 'Ь': 73, 'L': 74, 'x': 61}, {'Т': 118, 'T': 50, '.': 52, '#': 147}, {'.': 57, 'J': 50, '#': 37, 'G': 15, 'О': 73, 'O': 74, 'l': 61}, {'д': 57, '!': 50, 'О': 37, 'O': 15, '8': 147, 'l': 61}, {'‘': 57, 'O': 50, 'З': 37, 'S': 15, '#': 147, '0': 61}, {'‚': 57, '8': 50, 'Е': 37, 'G': 15, '1': 73, 'l': 74, '3': 61}, {'’': 57, ';': 50, 'Т': 37, '1': 15, '': 147, 'l': 61}, {'0': 57, '’': 50, '': 199, 'l': 61}, {'8': 57, 'H': 50, '': 199, '1': 61}, {';': 57, '': 310}, {"'": 57, '': 310}, {'П': 57, '': 310}], 'num16.jpg', "?'#O8#l", "?'#O8#l", "?'#O8#l", 'O8#l', 'I_ench_lines'], ['rusl --oem 0 --psm 11', [['ЁЬ#', 1262, 395, 3073, 1185, 57], ['О', 3284, 415, 3714, 1145, 92], ['8', 3946, 415, 4375, 1145, 89], ['#', 4586, 395, 5057, 1185, 67], ['1', 5268, 417, 5697, 1126, 63]], [{'Ь': 57, 'b': 50, 'Е': 37, 'H': 15, 'Ё': 73, '?': 74, 'r': 61}, {"'": 107, 'Т': 37, 'T': 15, 'Ь': 73, 'L': 74, 'x': 61}, {'Т': 118, 'T': 50, '.': 52, '#': 147}, {'.': 57, 'J': 50, '#': 37, 'G': 15, 'О': 73, 'O': 74, 'l': 61}, {'д': 57, '!': 50, 'О': 37, 'O': 15, '8': 147, 'l': 61}, {'‘': 57, 'O': 50, 'З': 37, 'S': 15, '#': 147, '0': 61}, {'‚': 57, '8': 50, 'Е': 37, 'G': 15, '1': 73, 'l': 74, '3': 61}, {'’': 57, ';': 50, 'Т': 37, '1': 15, '': 147, 'l': 61}, {'0': 57, '’': 50, '': 199, 'l': 61}, {'8': 57, 'H': 50, '': 199, '1': 61}, {';': 57, '': 310}, {"'": 57, '': 310}, {'П': 57, '': 310}], 'num16.jpg', "?'#O8#l", "?'#O8#l", "?'#O8#l", 'O8#l', 'I_ench_lines'], ['engl --oem 0 --psm 11', [['?L#', 1262, 395, 3073, 1185, 58], ['O', 3284, 415, 3714, 1145, 92], ['8', 3946, 415, 4375, 1145, 92], ['#', 4586, 395, 5057, 1185, 63], ['l', 5268, 417, 5697, 1126, 67]], [{'Ь': 57, 'b': 50, 'Е': 37, 'H': 15, 'Ё': 73, '?': 74, 'r': 61}, {"'": 107, 'Т': 37, 'T': 15, 'Ь': 73, 'L': 74, 'x': 61}, {'Т': 118, 'T': 50, '.': 52, '#': 147}, {'.': 57, 'J': 50, '#': 37, 'G': 15, 'О': 73, 'O': 74, 'l': 61}, {'д': 57, '!': 50, 'О': 37, 'O': 15, '8': 147, 'l': 61}, {'‘': 57, 'O': 50, 'З': 37, 'S': 15, '#': 147, '0': 61}, {'‚': 57, '8': 50, 'Е': 37, 'G': 15, '1': 73, 'l': 74, '3': 61}, {'’': 57, ';': 50, 'Т': 37, '1': 15, '': 147, 'l': 61}, {'0': 57, '’': 50, '': 199, 'l': 61}, {'8': 57, 'H': 50, '': 199, '1': 61}, {';': 57, '': 310}, {"'": 57, '': 310}, {'П': 57, '': 310}], 'num16.jpg', "?'#O8#l", "?'#O8#l", "?'#O8#l", 'O8#l', 'I_ench_lines'], ['nums_4m_v2_0.05_105926 --oem 3 --psm 11', [['rxТ', 1262, 395, 3073, 1185, 0], ['ll', 2425, 391, 3010, 1392, 52], ['0', 3284, 415, 3714, 1145, 97], ['3', 3946, 415, 4375, 1145, 67], ['ll', 4586, 395, 5057, 1185, 58], ['1', 5268, 417, 5697, 1126, 96]], [{'Ь': 57, 'b': 50, 'Е': 37, 'H': 15, 'Ё': 73, '?': 74, 'r': 61}, {"'": 107, 'Т': 37, 'T': 15, 'Ь': 73, 'L': 74, 'x': 61}, {'Т': 118, 'T': 50, '.': 52, '#': 147}, {'.': 57, 'J': 50, '#': 37, 'G': 15, 'О': 73, 'O': 74, 'l': 61}, {'д': 57, '!': 50, 'О': 37, 'O': 15, '8': 147, 'l': 61}, {'‘': 57, 'O': 50, 'З': 37, 'S': 15, '#': 147, '0': 61}, {'‚': 57, '8': 50, 'Е': 37, 'G': 15, '1': 73, 'l': 74, '3': 61}, {'’': 57, ';': 50, 'Т': 37, '1': 15, '': 147, 'l': 61}, {'0': 57, '’': 50, '': 199, 'l': 61}, {'8': 57, 'H': 50, '': 199, '1': 61}, {';': 57, '': 310}, {"'": 57, '': 310}, {'П': 57, '': 310}], 'num16.jpg', "?'#O8#l", "?'#O8#l", "?'#O8#l", 'O8#l', 'I_ench_lines']]


pd.DataFrame(l)

,0,1,2,3,4,5,6,7,8
0,num_ssr eng,"[[Ь'Т.д‘‚’08;'П, 78, 32, 205, 55, 57]]","[{'Ь': 57, 'b': 50, 'Е': 37, 'H': 15, 'Ё': 73,...",num16.jpg,?'#O8#l,?'#O8#l,?'#O8#l,O8#l,I_ench_lines
1,num_ssr rus,"[[Ь'Т.д‘‚’08;'П, 78, 32, 205, 55, 57]]","[{'Ь': 57, 'b': 50, 'Е': 37, 'H': 15, 'Ё': 73,...",num16.jpg,?'#O8#l,?'#O8#l,?'#O8#l,O8#l,I_ench_lines
2,rus --oem 3 --psm 11,"[[ЕТ., 1262, 395, 3073, 1185, 8], [#, 2842, 39...","[{'Ь': 57, 'b': 50, 'Е': 37, 'H': 15, 'Ё': 73,...",num16.jpg,?'#O8#l,?'#O8#l,?'#O8#l,O8#l,I_ench_lines
3,eng --oem 3 --psm 11,"[[HT., 1262, 395, 3073, 1185, 0], [G, 2676, 39...","[{'Ь': 57, 'b': 50, 'Е': 37, 'H': 15, 'Ё': 73,...",num16.jpg,?'#O8#l,?'#O8#l,?'#O8#l,O8#l,I_ench_lines
4,rusl --oem 0 --psm 11,"[[ЁЬ#, 1262, 395, 3073, 1185, 57], [О, 3284, 4...","[{'Ь': 57, 'b': 50, 'Е': 37, 'H': 15, 'Ё': 73,...",num16.jpg,?'#O8#l,?'#O8#l,?'#O8#l,O8#l,I_ench_lines
5,engl --oem 0 --psm 11,"[[?L#, 1262, 395, 3073, 1185, 58], [O, 3284, 4...","[{'Ь': 57, 'b': 50, 'Е': 37, 'H': 15, 'Ё': 73,...",num16.jpg,?'#O8#l,?'#O8#l,?'#O8#l,O8#l,I_ench_lines
6,nums_4m_v2_0.05_105926 --oem 3 --psm 11,"[[rxТ, 1262, 395, 3073, 1185, 0], [ll, 2425, 3...","[{'Ь': 57, 'b': 50, 'Е': 37, 'H': 15, 'Ё': 73,...",num16.jpg,?'#O8#l,?'#O8#l,?'#O8#l,O8#l,I_ench_lines


In [20]:
from varname import nameof

print(nameof(l))

l


In [78]:
import numpy as np
label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-INNKPP', 'I-INNKPP', 'B-RSKS', 'I-RSKS',
                  'B-STAT', 'I-STAT']
sp = ""
spans = []
tags = []
i = 0
conj = '##'
for idx,t in enumerate(nlp_res):
    li = int(t['entity'].split('_')[-1])
    if t['word'][:2] == conj:
        sp += t['word'][2:]
        tags.append([label_list[li], [t['start'],t['end']],t['score']])
#         print(t)
    else:
        sp += r' {}'.format(t['word'])
        tags.append([label_list[li], [t['start'],t['end']],t['score']])
    if idx < len(nlp_res) - 1:
        if nlp_res[idx + 1]['word'][:2] != conj:
            c,ft = make_coord_tags(tags) 
            sp = dict(word=sp.strip(), coord=c, tags=ft)
            spans.append(sp)
            sp = ""
            tags = []


In [77]:
def make_coord_tags(tags):
    coord = [tags[0][1][0], tags[-1][1][1]]
    ft = []
    sorted_tags = sorted(tags, key=lambda k: k[2], reverse=True)
    st = set(t[0] for t in sorted_tags)
    if len(st) > 1:
        ft = [[tt[0],tt[2]] for tt in sorted_tags]
    else:
        ft = [sorted_tags[0]]
    return coord, ft
        
    
# [np.argmax()]

In [160]:
spans =  
st = {'word': '', 'coord': [], 'tags': ['', 0]}
ent = []
result = []
for i, enti in enumerate(spans):
    e0 = enti['tags'][0][0]
    #     print(enti)
    if e0 in ['I', 'O']:
        if st['tags'][0] == enti['tags'][0]:
            st['word'] += ' ' + enti['word']
            st['coord'] = [st['coord'][0], enti['coord'][1]]
            st['tags'] = enti['tags']
        else:
            if len(st['word']) > 0:
                st['word'] = st['word'].strip()
                result.append(st)
            st = enti
    if e0 in ['B']:
        if len(st['word']) > 0:
            result.append(st)
            st = enti
        else:
            result.append(enti)
if len(st['word']) > 0:
    if len(result) == 0:
        result.append(st)
    elif st['tags'][0] != result[-1]['tags'][0]:
        result.append(st)

TypeError: list indices must be integers or slices, not str

In [100]:
 nlp_res = [{'word': 'инвестор', 'score': 0.9951106309890747, 'entity': 'LABEL_0', 'index': 1, 'start': 0, 'end': 8},
               {'word': 'о', 'score': 0.9450922012329102, 'entity': 'LABEL_3', 'index': 2, 'start': 9, 'end': 10},
               {'word': '##оо', 'score': 0.6366572499275208, 'entity': 'LABEL_2', 'index': 3, 'start': 10, 'end': 12},
               {'word': 'дю', 'score': 0.8052636981010437, 'entity': 'LABEL_0', 'index': 4, 'start': 13, 'end': 15}, {'word': '##ка', 'score': 0.7681756019592285, 'entity': 'LABEL_0', 'index': 5, 'start': 15, 'end': 17}, {'word': 'инн', 'score': 0.9823566675186157, 'entity': 'LABEL_0', 'index': 6, 'start': 18, 'end': 21}, {'word': 'к', 'score': 0.9696914553642273, 'entity': 'LABEL_0', 'index': 7, 'start': 22, 'end': 23}, {'word': '##пп', 'score': 0.9289431571960449, 'entity': 'LABEL_0', 'index': 8, 'start': 23, 'end': 25}, {'word': '564', 'score': 0.9995160698890686, 'entity': 'LABEL_7', 'index': 9, 'start': 26, 'end': 29}, {'word': '##500', 'score': 0.9988665580749512, 'entity': 'LABEL_7', 'index': 10, 'start': 29, 'end': 32}, {'word': '##39', 'score': 0.9993165731430054, 'entity': 'LABEL_7', 'index': 11, 'start': 32, 'end': 34}, {'word': '##4', 'score': 0.9999649524688721, 'entity': 'LABEL_7', 'index': 12, 'start': 34, 'end': 35}, {'word': '/', 'score': 0.9941858649253845, 'entity': 'LABEL_0', 'index': 13, 'start': 36, 'end': 37}, {'word': '231', 'score': 0.9995736479759216, 'entity': 'LABEL_7', 'index': 14, 'start': 38, 'end': 41}, {'word': '##201', 'score': 0.9999172687530518, 'entity': 'LABEL_7', 'index': 15, 'start': 41, 'end': 44}, {'word': '##001', 'score': 0.9998979568481445, 'entity': 'LABEL_7', 'index': 16, 'start': 44, 'end': 47}, {'word': '624', 'score': 0.9969303011894226, 'entity': 'LABEL_5', 'index': 17, 'start': 48, 'end': 51}, {'word': '##13', 'score': 0.9774559736251831, 'entity': 'LABEL_5', 'index': 18, 'start': 51, 'end': 53}, {'word': '##4', 'score': 0.9976329207420349, 'entity': 'LABEL_5', 'index': 19, 'start': 53, 'end': 54}, {'word': ',', 'score': 0.9588570594787598, 'entity': 'LABEL_6', 'index': 20, 'start': 54, 'end': 55}, {'word': 'сверд', 'score': 0.937218964099884, 'entity': 'LABEL_6', 'index': 21, 'start': 56, 'end': 61}, {'word': '##ловская', 'score': 0.9956352114677429, 'entity': 'LABEL_6', 'index': 22, 'start': 61, 'end': 68}, {'word': 'обл', 'score': 0.9999185800552368, 'entity': 'LABEL_6', 'index': 23, 'start': 69, 'end': 72}, {'word': ',', 'score': 0.9999923706054688, 'entity': 'LABEL_6', 'index': 24, 'start': 72, 'end': 73}, {'word': 'г', 'score': 0.9999490976333618, 'entity': 'LABEL_6', 'index': 25, 'start': 74, 'end': 75}, {'word': 'ново', 'score': 0.9959588050842285, 'entity': 'LABEL_6', 'index': 26, 'start': 76, 'end': 80}, {'word': '##уральск', 'score': 0.9996302127838135, 'entity': 'LABEL_6', 'index': 27, 'start': 80, 'end': 87}, {'word': ',', 'score': 0.9999957084655762, 'entity': 'LABEL_6', 'index': 28, 'start': 87, 'end': 88}, {'word': 'ул', 'score': 0.9999998211860657, 'entity': 'LABEL_6', 'index': 29, 'start': 89, 'end': 91}, {'word': 'чу', 'score': 0.9999946355819702, 'entity': 'LABEL_6', 'index': 30, 'start': 92, 'end': 94}, {'word': '##рина', 'score': 0.9999701380729675, 'entity': 'LABEL_6', 'index': 31, 'start': 94, 'end': 98}, {'word': ',', 'score': 1.0, 'entity': 'LABEL_6', 'index': 32, 'start': 98, 'end': 99}, {'word': 'д', 'score': 1.0, 'entity': 'LABEL_6', 'index': 33, 'start': 100, 'end': 101}, {'word': '12', 'score': 0.9999998807907104, 'entity': 'LABEL_6', 'index': 34, 'start': 102, 'end': 104}, {'word': '/', 'score': 0.9999989867210388, 'entity': 'LABEL_6', 'index': 35, 'start': 104, 'end': 105}, {'word': '2', 'score': 0.9999957084655762, 'entity': 'LABEL_6', 'index': 36, 'start': 105, 'end': 106}, {'word': ',', 'score': 0.9999526143074036, 'entity': 'LABEL_6', 'index': 37, 'start': 106, 'end': 107}, {'word': 'оф', 'score': 0.9999685883522034, 'entity': 'LABEL_6', 'index': 38, 'start': 108, 'end': 110}, {'word': '20', 'score': 0.9997296333312988, 'entity': 'LABEL_6', 'index': 39, 'start': 111, 'end': 113}, {'word': 'р', 'score': 0.5355002284049988, 'entity': 'LABEL_0', 'index': 40, 'start': 114, 'end': 115}, {'word': '##с', 'score': 0.5002505779266357, 'entity': 'LABEL_0', 'index': 41, 'start': 115, 'end': 116}, {'word': '444', 'score': 0.9877079725265503, 'entity': 'LABEL_9', 'index': 42, 'start': 117, 'end': 120}, {'word': '##68', 'score': 0.9969617128372192, 'entity': 'LABEL_9', 'index': 43, 'start': 120, 'end': 122}, {'word': '##25', 'score': 0.9986138343811035, 'entity': 'LABEL_9', 'index': 44, 'start': 122, 'end': 124}, {'word': '##37', 'score': 0.9994258880615234, 'entity': 'LABEL_9', 'index': 45, 'start': 124, 'end': 126}, {'word': '##34', 'score': 0.9997733235359192, 'entity': 'LABEL_9', 'index': 46, 'start': 126, 'end': 128}, {'word': '##65', 'score': 0.9997557997703552, 'entity': 'LABEL_9', 'index': 47, 'start': 128, 'end': 130}, {'word': '##94', 'score': 0.9994229078292847, 'entity': 'LABEL_9', 'index': 48, 'start': 130, 'end': 132}, {'word': '##94', 'score': 0.9902714490890503, 'entity': 'LABEL_9', 'index': 49, 'start': 132, 'end': 134}, {'word': '##40', 'score': 0.974387526512146, 'entity': 'LABEL_9', 'index': 50, 'start': 134, 'end': 136}, {'word': '##8', 'score': 0.9457467198371887, 'entity': 'LABEL_9', 'index': 51, 'start': 136, 'end': 137}, {'word': 'филиал', 'score': 0.960223913192749, 'entity': 'LABEL_0', 'index': 52, 'start': 138, 'end': 144}, {'word': 'банка', 'score': 0.9804496169090271, 'entity': 'LABEL_0', 'index': 53, 'start': 145, 'end': 150}, {'word': 'п', 'score': 0.8649019002914429, 'entity': 'LABEL_0', 'index': 54, 'start': 151, 'end': 152}, {'word': '-', 'score': 0.9643561244010925, 'entity': 'LABEL_0', 'index': 55, 'start': 152, 'end': 153}, {'word': 'щ', 'score': 0.9733700752258301, 'entity': 'LABEL_0', 'index': 56, 'start': 153, 'end': 154}, {'word': '##ик', 'score': 0.9066823720932007, 'entity': 'LABEL_0', 'index': 57, 'start': 154, 'end': 156}, {'word': 'акционерное', 'score': 0.9679957032203674, 'entity': 'LABEL_3', 'index': 58, 'start': 157, 'end': 168}, {'word': 'общество', 'score': 0.8897599577903748, 'entity': 'LABEL_4', 'index': 59, 'start': 169, 'end': 177}, {'word': 'коммерчески', 'score': 0.9034683108329773, 'entity': 'LABEL_4', 'index': 60, 'start': 178, 'end': 189}, {'word': '##и', 'score': 0.8095888495445251, 'entity': 'LABEL_4', 'index': 61, 'start': 189, 'end': 190}, {'word': 'банк', 'score': 0.9881073236465454, 'entity': 'LABEL_4', 'index': 62, 'start': 191, 'end': 195}, {'word': '"', 'score': 0.9711835384368896, 'entity': 'LABEL_4', 'index': 63, 'start': 196, 'end': 197}, {'word': 'куба', 'score': 0.9883135557174683, 'entity': 'LABEL_4', 'index': 64, 'start': 197, 'end': 201}, {'word': '##нс', 'score': 0.9092617630958557, 'entity': 'LABEL_4', 'index': 65, 'start': 201, 'end': 203}, {'word': '##ки', 'score': 0.9342242479324341, 'entity': 'LABEL_4', 'index': 66, 'start': 203, 'end': 205}, {'word': '##и', 'score': 0.9919331073760986, 'entity': 'LABEL_4', 'index': 67, 'start': 205, 'end': 206}, {'word': 'торгов', 'score': 0.9903615117073059, 'entity': 'LABEL_4', 'index': 68, 'start': 207, 'end': 213}, {'word': '##ы', 'score': 0.956931471824646, 'entity': 'LABEL_4', 'index': 69, 'start': 213, 'end': 214}, {'word': '##и', 'score': 0.9914941191673279, 'entity': 'LABEL_4', 'index': 70, 'start': 214, 'end': 215}, {'word': 'банк', 'score': 0.9995408654212952, 'entity': 'LABEL_4', 'index': 71, 'start': 216, 'end': 220}, 
            {'word': '"', 'score': 0.9950947761535645, 'entity': 'LABEL_4', 'index': 72, 'start': 220, 'end': 221}]

{'B': [('B-ORG', 1)],
 'I': [('I-PER', 1)],
 'O': [('', 0)],
 'Overall': [('B-ORG', 1), ('I-PER', 1)]}

In [177]:
spans =[{'word': 'Инвестор', 'coord': [0, 8], 'tags': ['O', 0.9951106309890747]}, 
        {'word': 'ООО', 'coord': [9, 12], 'tags': ['B-ORG', 0.9450922012329102]}, {'word': 'ДЮКА ИНН КПП', 'coord': [13, 25], 'tags': ['O', 0.9696914553642273]}, {'word': '564500394', 'coord': [26, 35], 'tags': ['B-INNKPP', 0.9999649524688721]}, {'word': '/', 'coord': [36, 37], 'tags': ['O', 0.9941858649253845]}, {'word': '231201001', 'coord': [38, 47], 'tags': ['B-INNKPP', 0.9999172687530518]}, {'word': '624134', 'coord': [48, 54], 'tags': ['B-LOC', 0.9976329207420349]}, {'word': ', Свердловская обл , г Новоуральск , ул Чурина , д 12 / 2 , оф 20', 'coord': [54, 113], 'tags': ['I-LOC', 0.9997296333312988]}, {'word': 'рс', 'coord': [114, 116], 'tags': ['O', 0.5355002284049988]}, {'word': '44468253734659494408', 'coord': [117, 137], 'tags': ['B-RSKS', 0.9997733235359192]}, {'word': 'Филиал банка П - щик', 'coord': [138, 156], 'tags': ['O', 0.9733700752258301]}, {'word': 'АКЦИОНЕРНОЕ', 'coord': [157, 168], 'tags': ['B-ORG', 0.9679957032203674]}, {'word': 'ОБЩЕСТВО КОММЕРЧЕСКИЙ БАНК " КУБАНСКИЙ ТОРГОВЫЙ БАНК', 'coord': [169, 220], 'tags': ['I-ORG', 0.9995408654212952]}]

result = []

# Нормализуем Теги "B-", если слово начинается с тега "i-"
# и перед ним нет аналогичного тега "B-", то присвоить ему тег "B-"
for i, enti in enumerate(spans):
    e0 = enti['tags'][0][0]
    # if 'литлтон' in tokens[i]:
    #     print(tokens[i])
    if e0 == 'I':
        if i == 0:
            enti['tags'][0] = ['tags'][0].replace('I-', 'B-')
            result.append(enti)
        else:
            if spans[i - 1]['tags'][0][2:] == enti['tags'][0][2:]:
                result.append(enti)
            else:
                enti['tags'][0] = ['tags'][0].replace('I-', 'B-')
                result.append(enti)
    else:
        result.append(enti)
res_spans = result
# tokens, entities = zip(*result)
st = {'word': '', 'coord': [], 'tags': ['', 0]}
result = []

for i, enti in enumerate(res_spans):
    e0 = enti['tags'][0][0]
#     print(e0)
    if e0 in ['B']:
        if i < len(res_spans)-1 and res_spans[i + 1]['tags'][0][0] == 'I':
            if res_spans[i + 1]['tags'][0][2:] == enti['tags'][0][2:]:
                st['word'] += ' ' + enti['word']
                if len(st['coord']) > 0:
                        st['coord'] = [st['coord'][0], enti['coord'][1]]
                else:
                    st['coord'] = [enti['coord'][0], enti['coord'][1]]
                st['tags'] = enti['tags']
                continue
        else:
            if len(st['word']) > 0:
                if st['tags'][0][2:] == enti[2:]:
                    st['word'] += ' ' + enti['word']
                    if len(st['coord']) > 0:
                        st['coord'] = [st['coord'][0], enti['coord'][1]]
                    else: 
                        st['coord'] = [enti['coord'][0], enti['coord'][1]]
                    st['tags'] = enti['tags']
                    st = {'word': '', 'coord': [], 'tags': ['', 0]}
            else:
                result.append(enti)
    elif e0 in ['I']:
        if len(st['word']) > 0:
            print(enti)
            if st['tags'][0][2:] == enti['tags'][0][2:]:
                st['word'] += ' ' + enti['word']
                if len(st['coord']) > 0:
                    st['coord'] = [st['coord'][0], enti['coord'][1]]
                else: 
                    st['coord'] = [enti['coord'][0], enti['coord'][1]]
#                 st['tags'] = enti['tags']
                result.append(st)
                st = {'word': '', 'coord': [], 'tags': ['', 0]}
            else:
                st['word'] = st['word'].strip()
                result.append(st)
                st = {'word': '', 'coord': [], 'tags': ['', 0]}
            continue
        else:
            if res_spans[i + 1]['tags'][0][0] == 'B':
                if res_spans[i + 1]['tags'][0][2:] == enti['tags'][0][2:]:
                    st['word'] += ' ' + enti['word']
                    
                    if len(st['coord']) > 0:
                        st['coord'] = [st['coord'][0], res_spans[i + 1]['coord'][1]]
                    else: 
                        st['coord'] = [res_spans[i + 1]['coord'][0], res_spans[i + 1]['coord'][1]]
                    st['tags'] = enti['tags']
                    st['tags'] = res_spans[i + 1]['tags']
                    continue

    elif e0 in ['O']:
        result.append(enti)
        continue

{'word': ', Свердловская обл , г Новоуральск , ул Чурина , д 12 / 2 , оф 20', 'coord': [54, 113], 'tags': ['I-LOC', 0.9997296333312988]}
{'word': 'ОБЩЕСТВО КОММЕРЧЕСКИЙ БАНК " КУБАНСКИЙ ТОРГОВЫЙ БАНК', 'coord': [169, 220], 'tags': ['I-ORG', 0.9995408654212952]}


In [178]:
result

[{'word': 'Инвестор', 'coord': [0, 8], 'tags': ['O', 0.9951106309890747]},
 {'word': 'ООО', 'coord': [9, 12], 'tags': ['B-ORG', 0.9450922012329102]},
 {'word': 'ДЮКА ИНН КПП',
  'coord': [13, 25],
  'tags': ['O', 0.9696914553642273]},
 {'word': '564500394',
  'coord': [26, 35],
  'tags': ['B-INNKPP', 0.9999649524688721]},
 {'word': '/', 'coord': [36, 37], 'tags': ['O', 0.9941858649253845]},
 {'word': '231201001',
  'coord': [38, 47],
  'tags': ['B-INNKPP', 0.9999172687530518]},
 {'word': ' 624134 , Свердловская обл , г Новоуральск , ул Чурина , д 12 / 2 , оф 20',
  'coord': [48, 113],
  'tags': ['B-LOC', 0.9976329207420349]},
 {'word': 'рс', 'coord': [114, 116], 'tags': ['O', 0.5355002284049988]},
 {'word': '44468253734659494408',
  'coord': [117, 137],
  'tags': ['B-RSKS', 0.9997733235359192]},
 {'word': 'Филиал банка П - щик',
  'coord': [138, 156],
  'tags': ['O', 0.9733700752258301]},
 {'word': ' АКЦИОНЕРНОЕ ОБЩЕСТВО КОММЕРЧЕСКИЙ БАНК " КУБАНСКИЙ ТОРГОВЫЙ БАНК',
  'coord': [157, 22

In [144]:
text = 'Инвестор ООО ДЮКА ИНН КПП 564500394 / 231201001 624134, Свердловская обл, г Новоуральск, ул Чурина, д 12/2, оф 20 \
рс 44468253734659494408 Филиал банка П-щик АКЦИОНЕРНОЕ ОБЩЕСТВО КОММЕРЧЕСКИЙ БАНК "КУБАНСКИЙ ТОРГОВЫЙ БАНК" '

In [146]:
text[69:72]

'обл'

In [134]:
[['инвестор', 'O'], ['ооо', 'B-ORG'], ['дюка инн кпп', 'O'], ['564500394', 'B-INNKPP'], ['/', 'O'], ['231201001', 'B-INNKPP'], [' 624134 , свердловская обл , г новоуральск , ул чурина , д 12 / 2 , оф 20', 'B-LOC'], ['рс', 'O'], ['44468253734659494408', 'B-RSKS'], 
 ['филиал банка п - щик', 'O'], [' акционерное общество коммерческии банк " кубанскии торговыи банк "', 'B-ORG']]

[{'word': 'Инвестор', 'coord': [0, 8], 'tags': ['O', 0.9951106309890747]},
 {'word': 'ООО', 'coord': [9, 12], 'tags': ['B-ORG', 0.5450922012329102]},
 {'word': 'ДЮКА', 'coord': [13, 17], 'tags': ['O', 0.8052636981010437]},
 {'word': 'ИНН', 'coord': [18, 21], 'tags': ['O', 0.9823566675186157]},
 {'word': 'КПП', 'coord': [22, 25], 'tags': ['O', 0.9696914553642273]},
 {'word': '564500394',
  'coord': [26, 35],
  'tags': ['B-INNKPP', 0.9999649524688721]},
 {'word': '/', 'coord': [36, 37], 'tags': ['O', 0.9941858649253845]},
 {'word': '231201001',
  'coord': [38, 47],
  'tags': ['B-INNKPP', 0.9999172687530518]},
 {'word': '624134', 'coord': [48, 54], 'tags': ['B-LOC', 0.9976329207420349]},
 {'word': ',', 'coord': [54, 55], 'tags': ['B-LOC', 0.9588570594787598]},
 {'word': 'Свердловская',
  'coord': [56, 68],
  'tags': ['B-LOC', 0.9956352114677429]},
 {'word': 'обл', 'coord': [69, 72], 'tags': ['B-LOC', 0.9999185800552368]},
 {'word': ',', 'coord': [72, 73], 'tags': ['B-LOC', 0.9999923706054